In [1]:
import LoadData
import Similarity
from ipywidgets import widgets, Layout
from IPython.display import clear_output #Used to clear the output

#All the Gui Elements
style = {'description_width': 'initial'}
loadDataButton = widgets.Button(description = "Load Data", disabled = False, style = style)
compMoviesOrUsers = widgets.RadioButtons(options=['Users', 'Movies'], description = 'Compare', diabled = True, style = style)
userIDText = widgets.BoundedIntText(min=1,max=10,step=1,description='UserID:',disabled=True, )
movieNameDropdown = widgets.Dropdown(options=[], description = 'Target Movie:', disabled = True, style = style)
recMovieOrUserRadio = widgets.RadioButtons(options=['Movies', 'Users'],description='Recommend',disabled=True, style = style)
compareMoviesByRadio = widgets.RadioButtons(options=['Genres', 'Users'],description='Compare Movies By',disabled=True, style = style)
amountText = widgets.BoundedIntText(min=1,max=20,step=1,description='Amount:',disabled=True, style = style)
simFunc = widgets.Dropdown(options=['Euclidean', 'Cosine', 'Pearson', 'Jaccard', 'Manhatten', 'Custom'],description='Similarity Function',disabled=False, style = style)
calculateButton = widgets.Button(description = "Start", disabled = True, style = style)
results = widgets.Textarea(value='',description='Recommendations',disabled=True, style = style, layout = Layout(width = '50%', height = '200px'))


#DataSets:

userData = None
movieData = None

#GUI Functions

def onLoadButtonClicked(b):
    
    global userData
    global movieData
    
    data = LoadData.loadData()
    userData = data[0]
    userData.sort(key = lambda x: int(x.getID()))
    movieData = data[1]
    tempList = list()
    for movie in movieData:
        tempList.append(movie.getName())
    movieNameDropdown.options = tuple(tempList)
    userIDText.max = len(userData) -1
    loadDataButton.disabled = True
    enableMainGUI(True)

loadDataButton.on_click(onLoadButtonClicked)



def onCalcButtonClicked(b):
    
    calculateButton.disabled = True
    calculateButton.description = "Calculating"
    
    recMovies = False
    target = None
    
    if compMoviesOrUsers.value == 'Movies':
        recMovies = True
        targetName = movieNameDropdown.value
        for movie in movieData:
            if movie.getName() == targetName:
                target = movie
                break
    else:
        target = userData[int(userIDText.value)-1]
    
    compMoviesBy = True
    if compareMoviesByRadio.value == 'Users':
        compMoviesBy = False
    simFuncVal = 0
    if simFunc.value == 'Cosine':
        simFuncVal = 1
    elif simFunc.value == "Pearson":
        simFuncVal = 2
    elif simFunc.value == 'Jaccard':
        simFuncVal = 3
    elif simFunc.value == 'Manhatten':
        simFuncVal = 4
        
    recUsers = False
    if recMovieOrUserRadio.value == 'Users':
        recUsers = True
        
    if simFunc.value == 'Custom':       
        rec  = Similarity.customSimilarity(target, userData, movieData, int(amountText.value), recUsers)
    else:
        if recMovies:
            rec = Similarity.similarMovies(target, movieData, int(amountText.value), simFuncVal, compMoviesBy)
        else:
            rec = Similarity.recommendMovies(target, userData, movieData, int(amountText.value), simFuncVal, compMoviesBy, recUsers)
    
    tempString = ""
    for recommendation in rec:
        tempString += recommendation[0] + "\n"
    
    results.value = tempString
    displayGUI()
    display(results)
    calculateButton.disabled = False
    calculateButton.description = "Start"
    
calculateButton.on_click(onCalcButtonClicked)

def onTraitChangeCompMovieOrUserRadio(change):
    pos = change['new']
    if int(pos) == 1:
        userIDText.disabled = True
        recMovieOrUserRadio.disabled = True
        movieNameDropdown.disabled = False
        simFunc.options = ['Euclidean', 'Cosine', 'Pearson', 'Jaccard', 'Manhatten']
    if int(pos) == 0:
        userIDText.disabled = False
        recMovieOrUserRadio.disabled = False
        movieNameDropdown.disabled = True
        simFunc.options = ['Euclidean', 'Cosine', 'Pearson', 'Jaccard', 'Manhatten', 'Custom']
        
compMoviesOrUsers.observe(onTraitChangeCompMovieOrUserRadio, names='index')

def onTraitChangeRecMovieOrUserRadio(change):
    pos = change['new']
    if int(pos) == 0:
        compareMoviesByRadio.disabled = False
    else:
        compareMoviesByRadio.disabled = True
    
recMovieOrUserRadio.observe(onTraitChangeRecMovieOrUserRadio, names='index')

def displayGUI():
    clear_output()
    display(loadDataButton)
    display(compMoviesOrUsers)
    display(userIDText)
    display(movieNameDropdown)
    display(recMovieOrUserRadio)
    display(compareMoviesByRadio)
    display(amountText)
    display(simFunc)
    display(calculateButton)
    

def enableMainGUI(enable = True):
    compMoviesOrUsers.disabled = not enable
    userIDText.disabled = not enable
    movieNameDropdown.disabled = not enable
    recMovieOrUserRadio.disabled = not enable
    compareMoviesByRadio.disabled = not enable
    amountText.disabled = not enable
    simFunc.disabled = not enable
    calculateButton.disabled = not enable
    if compMoviesOrUsers.value == 'Users':
        movieNameDropdown.disabled = True
    displayGUI()
    
#Load Data
display(loadDataButton)








Button(description='Load Data', disabled=True, style=ButtonStyle())

RadioButtons(description='Compare', options=('Users', 'Movies'), style=DescriptionStyle(description_width='ini…

BoundedIntText(value=8, description='UserID:', max=942, min=1)

Dropdown(description='Target Movie:', disabled=True, options=('Toy Story', 'GoldenEye', 'Four Rooms', 'Get Sho…

RadioButtons(description='Recommend', options=('Movies', 'Users'), style=DescriptionStyle(description_width='i…

RadioButtons(description='Compare Movies By', options=('Genres', 'Users'), style=DescriptionStyle(description_…

BoundedIntText(value=5, description='Amount:', max=20, min=1, style=DescriptionStyle(description_width='initia…

Dropdown(description='Similarity Function', index=3, options=('Euclidean', 'Cosine', 'Pearson', 'Jaccard', 'Ma…

Button(description='Calculating', disabled=True, style=ButtonStyle())

Textarea(value='187\nVegas Vacation\nThe Sixth Man\nThe Substitute\nAbsolute Power\n', description='Recommenda…